In [28]:
import numpy as np
from pathlib import Path
from plyfile import *

p = Path() / 'data' / 'branch_only' / 'apple_1.npz'
arr = np.load(p)

def load_ptcloud(filepath: Path):
    data = np.load(filepath)
    res = {}
    for f in data.files:
        res[f] = data[f]
    return res

In [3]:
arr.files

['tree_id',
 'xyz',
 'rgb',
 'vector',
 'class_l',
 'skeleton_xyz',
 'skeleton_radii',
 'branch_id',
 'branch_parent_id',
 'branch_num_elements']

In [16]:
res = {}
for f in arr.files:
    res[f] = arr[f]
res

{'tree_id': array(1, dtype=int64),
 'xyz': array([[-0.72603982,  2.38983823, -2.05716025],
        [-0.73567971,  2.38677101, -2.05120432],
        [-0.83291875,  2.34286693, -1.82518052],
        ...,
        [ 0.41545837,  3.04860716,  2.05080537],
        [ 0.51956767,  3.18069251,  0.4061205 ],
        [ 0.52532258,  3.17778453,  0.44655121]]),
 'rgb': array([[0.07843137, 0.0745098 , 0.05098039],
        [0.07843137, 0.0745098 , 0.05098039],
        [0.10784314, 0.10392157, 0.07647059],
        ...,
        [0.28627451, 0.28627451, 0.2745098 ],
        [0.20392157, 0.20392157, 0.17254902],
        [0.06078431, 0.06078431, 0.04215686]]),
 'vector': array([[ 0.00743562,  0.00708866, -0.00726199],
        [ 0.01717192,  0.01031709, -0.01296163],
        [-0.00970441, -0.00027251, -0.00721502],
        ...,
        [-0.00351408, -0.0161798 ,  0.00470281],
        [ 0.02690023, -0.016608  ,  0.01317024],
        [ 0.02380085,  0.01994205, -0.00079837]], dtype=float32),
 'class_l': array

In [22]:
ptcloud = load_ptcloud(p)
for k, v in ptcloud.items():
    print(f'{k}: {v.shape}')

tree_id: ()
xyz: (117996, 3)
rgb: (117996, 3)
vector: (117996, 3)
class_l: (117996,)
skeleton_xyz: (41311, 3)
skeleton_radii: (41311, 1)
branch_id: (10742,)
branch_parent_id: (10742,)
branch_num_elements: (10742,)


In [26]:
ptcloud['tree_id']

array(1, dtype=int64)

---

In [1]:
from plyfile import *
from pathlib import Path
import numpy as np
import open3d as o3d
# p = Path('out/run1/saved_lines.ply')
p = Path('out/run2/saved_lines.ply')
plydata = PlyData.read(p)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
for elem in plydata.elements:
    print(elem.name)
    print(f'  - count: {elem.count}')
    print(f'  - shape: {elem.data.shape}')
    print(f'  - dtype: {elem.data.dtype}')
    print(f'  - props: ')
    for prop in elem.properties:
        print(f'    - {prop.name}: {prop.val_dtype}')
    


vertex
  - count: 17420
  - shape: (17420,)
  - dtype: [('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('red', 'u1'), ('green', 'u1'), ('blue', 'u1')]
  - props: 
    - x: f8
    - y: f8
    - z: f8
    - red: u1
    - green: u1
    - blue: u1
face
  - count: 33500
  - shape: (33500,)
  - dtype: [('vertex_indices', 'O')]
  - props: 
    - vertex_indices: u4


In [3]:
vertices = plydata['vertex']
x = vertices['x']
y = vertices['y']
z = vertices['z']
r = vertices['red']
g = vertices['green']
b = vertices['blue']

positions = np.stack([x, y, z], axis=1)
colors = np.stack([r, g, b], axis=1)

print(f'positions: {positions.shape}')
print(f'colors: {colors.shape}')

faces = np.stack(plydata['face']['vertex_indices'], axis=0)
print(f'faces: {faces.shape}')

positions: (17420, 3)
colors: (17420, 3)
faces: (33500, 3)


In [4]:
v3v_pos = o3d.utility.Vector3dVector(positions)
v3v_tri = o3d.utility.Vector3iVector(faces)

mesh = o3d.geometry.TriangleMesh(v3v_pos, v3v_tri)
o3d.visualization.draw_geometries([mesh])